In [ ]:
print("임승혁 수정wjd")

In [2]:
import os
import csv
import requests
import bs4
import pandas
import re
import time
import datetime
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import multiprocessing

In [3]:
now = datetime.datetime.now()
year = now.isoformat()[0:4]
month = now.isoformat()[5:7]
day = now.isoformat()[8:10]
date = year+month+day

In [4]:
now.isoformat()[0:10]

'2023-05-20'

In [8]:
url = f'https://finance.naver.com/research/company_list.naver\
?keyword=&brokerCode=&searchType=\
writeDate&writeFromDate=2018-01-01&writeToDate=2023-05-19&itemName=&itemCode=&x=0&y=0&page='
print(url)

https://finance.naver.com/research/company_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2018-01-01&writeToDate=2023-05-19&itemName=&itemCode=&x=0&y=0&page=


In [ ]:
print("민정")

In [ ]:
    link = soup.select('td.content li dt a')
    f_link_list = list(set([i.attrs['href'] for i in link]))
    link_list.append(f_link_list)
    addr = soup.find_all('a', {'class':'nclicks(fls.page)'})

    next_button = soup.find('a', {'class':'next'})
    if next_button:
        page += 1

    elif next_button is None:
        numlist = [i.get_text() for i in addr][1:]
        for page in numlist:
            page_url = url + str(page)
            response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
            soup = bs4.BeautifulSoup(response.text, 'html.parser')
            link = soup.select('td.content li dt a')
            f_link_list = list(set([i.attrs['href'] for i in link]))
            link_list.append(f_link_list)
        break

In [6]:
page = 1
next_link_text = "다음"
stock_name_list = []
link_list = []

page_url = url + str(page)
response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
soup = bs4.BeautifulSoup(response.text, 'html.parser')
last_page = soup.select('table tr td a')[-6]['href'].split('/')[-1].split('=')[-1]

for _ in tqdm(range(int(last_page))):
    
    page_url = url + str(page)
    response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    stock_name = soup.select('table a.stock_item')[0:30]
    link = soup.select('table td:nth-child(2) a')[:-6]
    for i in range(len(soup.select('table a[target]'))):
        stock_name_list.append(stock_name[i].text)
        link_list.append('https://finance.naver.com/research/' + link[i]['href'])
    page += 1

100%|████████████████████████████████████████████████████████████████████████████████| 721/721 [03:42<00:00,  3.24it/s]


In [7]:
len(link_list)

21619

In [108]:
options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
options.add_argument("headless")
driver = webdriver.Chrome(options=options)

driver.get(link_list[0])
text = driver.find_element(By.XPATH, '//*[@id="contentarea_left"]/div[2]/table/tbody/tr[1]/th[1]/p')
broker = text.text.split('|')[0]
time = text.text.split('|')[1]
view_cnt = ''.join(filter(str.isdigit, text.text.split('|')[2]))

text = driver.find_element(By.XPATH, '//*[@id="contentarea_left"]/div[2]/table/tbody/tr[1]/th[1]/span/em')
stock_name = text.text
text = driver.find_element(By.CSS_SELECTOR, 'th.view_sbj')
title = text.text

In [109]:
/html/body/div[3]/div[2]/div[2]/div[1]/div[2]/table/tbody/tr[1]/th[1]/text()

'SK스퀘어 Harvest and Return 30~50%\nDS투자증권|2023.05.18|조회 651'

In [ ]:


time_list = []
company_list = []
title_list = []
main_txt_list = []
http_link = []

options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
options.add_argument("headless")
driver = webdriver.Chrome(options=options)

for link in tqdm(link_list):
    try:
        driver.get(link)
        text = driver.find_element(By.XPATH, '//*[@id="ct"]/div[1]/div[3]/div[1]/div[1]/span')
        post_time = text.text
        text = driver.find_element(By.XPATH, '//*[@id="ct"]/div[1]/div[1]/a/img[1]')
        company = text.get_attribute('alt')
        text = driver.find_element(By.XPATH, '//*[@id="title_area"]/span')
        title = text.text
        text = driver.find_element(By.XPATH, '//*[@id="dic_area"]')
        main_txt = text.text

        time_list.append(post_time)
        company_list.append(company)
        title_list.append(title)
        main_txt_list.append(main_txt)
        http_link.append(link)

    except:
        continue
df = pd.DataFrame({'분류':index, '시간':time_list, '언론사':company_list, '제목':title_list, '본문':main_txt_list, '링크':http_link})
return df


In [6]:
csv_path = 'C:/myPyCode/news_csv/'

In [19]:
def link_parser(url_list : list, category : str) -> list:
    
    for url in tqdm(url_list):
        page = 1
        next_link_text = "다음"
        link_list = []
        while True:
            page_url = url + str(page)

            response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
            soup = bs4.BeautifulSoup(response.text, 'html.parser')
            link = soup.select('td.content li dt a')
            f_link_list = list(set([i.attrs['href'] for i in link]))
            link_list.append(f_link_list)
            addr = soup.find_all('a', {'class':'nclicks(fls.page)'})

            next_button = soup.find('a', {'class':'next'})
            if next_button:
                page += 1

            elif next_button is None:
                numlist = [i.get_text() for i in addr][1:]
                for page in numlist:
                    page_url = url + str(page)
                    response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
                    soup = bs4.BeautifulSoup(response.text, 'html.parser')
                    link = soup.select('td.content li dt a')
                    f_link_list = list(set([i.attrs['href'] for i in link]))
                    link_list.append(f_link_list)
                break
    
    link_list = [j for i in link_list for j in i]
    if old_link2 == 0:
        total_link = link_list
        additional_link = list(set(total_link))
        df = pd.DataFrame({'링크':total_link})
        df.to_csv(csv_path + f'{category}.csv')
    else:
        total_link = old_link2 + link_list
        total_link = list(set(total_link))
        df = pd.DataFrame({'링크':total_link})
        df.to_csv(csv_path + f'{category}.csv')
        additional_link = [i for i in total_link if i not in old_link2]
    return additional_link


def get_txt(link_list : list, index : str):

    time_list = []
    company_list = []
    title_list = []
    main_txt_list = []
    http_link = []
    csv_path = 'C:/myPyCode/news_csv/'
    
    options = webdriver.ChromeOptions()
    # 창 숨기는 옵션 추가
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
        
    for link in tqdm(link_list):
        try:
            driver.get(link)
            text = driver.find_element(By.XPATH, '//*[@id="ct"]/div[1]/div[3]/div[1]/div[1]/span')
            post_time = text.text
            text = driver.find_element(By.XPATH, '//*[@id="ct"]/div[1]/div[1]/a/img[1]')
            company = text.get_attribute('alt')
            text = driver.find_element(By.XPATH, '//*[@id="title_area"]/span')
            title = text.text
            text = driver.find_element(By.XPATH, '//*[@id="dic_area"]')
            main_txt = text.text

            time_list.append(post_time)
            company_list.append(company)
            title_list.append(title)
            main_txt_list.append(main_txt)
            http_link.append(link)
            
        except:
            continue
    df = pd.DataFrame({'분류':index, '시간':time_list, '언론사':company_list, '제목':title_list, '본문':main_txt_list, '링크':http_link})
    return df

def main():
    start_time = time.time()
    if '*.csv' not in os.listdir(csv_path):
        politic_list_ = link_parser(politic_list, '정치')
        economy_list_ = link_parser(economy_list, '경제')
        society_list_ = link_parser(society_list, '사회')
        life_culture_list_ = link_parser(life_culture_list, '생활_문화')
        IT_science_list_ = link_parser(IT_science_list, 'IT_과학')
        world_list_ = link_parser(world_list, '세계')
        politic_df = get_txt(politic_list_, '정치')
        economy_df = get_txt(economy_list_, '경제')
        society_df = get_txt(society_list_, '사회')
        life_culture_df = get_txt(life_culture_list_, '생활_문화')
        IT_science_df = get_txt(IT_science_list_, 'IT_과학')
        world_df = get_txt(world_list_, '세계')
        news_df = pd.concat([politic_df, economy_df, society_df, life_culture_df, IT_science_df, world_df], axis = 0)
        news_df.to_csv(csv_path + 'news_df.csv')
        end_time = time.time()
        print('걸린 시간 :', (end_time - start_time) / 60, '분')
        print('크롤링 완료')
    else:
        politic_list_ = link_parser(politic_list, '정치')
        economy_list_ = link_parser(economy_list, '경제')
        society_list_ = link_parser(society_list, '사회')
        life_culture_list_ = link_parser(life_culture_list, '생활_문화')
        IT_science_list_ = link_parser(IT_science_list, 'IT_과학')
        world_list_ = link_parser(world_list, '세계')
        politic_df = get_txt(politic_list_, '정치')
        economy_df = get_txt(economy_list_, '경제')
        society_df = get_txt(society_list_, '사회')
        life_culture_df = get_txt(life_culture_list_, '생활_문화')
        IT_science_df = get_txt(IT_science_list_, 'IT_과학')
        world_df = get_txt(world_list_, '세계')
        news_df_old = pd.read_csv(csv_path + 'news_df.csv')
        news_df = pd.concat([news_df_old, politic_df, economy_df, society_df, life_culture_df, IT_science_df, world_df], axis = 0)
        news_df.to_csv(csv_path + 'news_df.csv')
        end_time = time.time()
        print('걸린 시간 :', (end_time - start_time) / 60, '분')
        print('크롤링 완료')

 25%|█████████████████████                                                               | 2/8 [00:19<00:57,  9.66s/it]


KeyboardInterrupt: 

In [ ]:

start_time = time.time()
pool = multiprocessing.Pool(processes=1) # 6개의 processes 사용
# politic_list_ = link_parser(politic_list, '정치')
# economy_list_ = link_parser(economy_list, '경제')
# society_list_ = link_parser(society_list, '사회')
# life_culture_list_ = link_parser(life_culture_list, '생활/문화')
# IT_science_list_ = link_parser(IT_science_list, 'IT/과학')
world_list_ = link_parser(world_list, '세계')
# pool.map(get_txt, politic_list_)
# pool.map(get_txt, economy_list_)
# pool.map(get_txt, society_list_)
# pool.map(get_txt, life_culture_list_)
# pool.map(get_txt, IT_science_list_)
pool.map(get_txt, world_list_)
pool.close()
pool.join()
end_time = time.time()
print('걸린 시간 :', (end_time - start_time) / 60, '분')
print('크롤링 완료')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.20s/it]


In [ ]:
start_time = time.time()
main()
end_time = time.time()
print('걸린 시간 :', (end_time - start_time) / 60, '분')
print('크롤링 완료')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.67s/it]


In [1]:
get_txt

NameError: name 'get_txt' is not defined

In [15]:
os.listdir(csv_path)

['IT_과학.csv',
 'news_df.csv',
 '경제.csv',
 '사회.csv',
 '생활_문화.csv',
 '세계.csv',
 '정치.csv']

In [17]:
file_list_csv = [file for file in os.listdir(csv_path) if file.endswith(".csv")]

['IT_과학.csv',
 'news_df.csv',
 '경제.csv',
 '사회.csv',
 '생활_문화.csv',
 '세계.csv',
 '정치.csv']

In [6]:
for url in tqdm(world_list):
    page = 1
    next_link_text = "다음"
    link_list = []
    while True:
        page_url = url + str(page)

        response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
        soup = bs4.BeautifulSoup(response.text, 'html.parser')
        link = soup.select('td.content li dt a')
        f_link_list = list(set([i.attrs['href'] for i in link]))
        link_list.append(f_link_list)
        addr = soup.find_all('a', {'class':'nclicks(fls.page)'})

        next_button = soup.find('a', {'class':'next'})
        if next_button:
            page += 1

        elif next_button is None:
            numlist = [i.get_text() for i in addr][1:]
            for page in numlist:
                page_url = url + str(page)
                response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
                soup = bs4.BeautifulSoup(response.text, 'html.parser')
                link = soup.select('td.content li dt a')
                f_link_list = list(set([i.attrs['href'] for i in link]))
                link_list.append(f_link_list)
            break

link_list = [j for i in link_list for j in i]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.55s/it]


In [7]:
time_list = []
company_list = []
title_list = []
main_txt_list = []
http_link = []
csv_path = 'C:/myPyCode/news_csv/'

if link_list[0][-3:] == '100':index = '정치'
if link_list[0][-3:] == '101':index = '경제'
if link_list[0][-3:] == '102':index = '사회'
if link_list[0][-3:] == '103':index = '생활/문화'
if link_list[0][-3:] == '104':index = '세계'
if link_list[0][-3:] == '105':index = 'IT/과학'

options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
options.add_argument("headless")
driver = webdriver.Chrome(options=options)

for link in tqdm(link_list[0:10]):
    try:
        driver.get(link)
        text = driver.find_element(By.XPATH, '//*[@id="ct"]/div[1]/div[3]/div[1]/div[1]/span')
        post_time = text.text
        text = driver.find_element(By.XPATH, '//*[@id="ct"]/div[1]/div[1]/a/img[1]')
        company = text.get_attribute('alt')
        text = driver.find_element(By.XPATH, '//*[@id="title_area"]/span')
        title = text.text
        text = driver.find_element(By.XPATH, '//*[@id="dic_area"]')
        main_txt = text.text

        time_list.append(post_time)
        company_list.append(company)
        title_list.append(title)
        main_txt_list.append(main_txt)
        http_link.append(link)

    except:
        continue
df = pd.DataFrame({'분류':index, '시간':time_list, '언론사':company_list, '제목':title_list, '본문':main_txt_list, '링크':http_link})

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.03s/it]


In [8]:
df

,분류,시간,언론사,제목,본문,링크
0,세계,2023.03.15. 오후 4:24,SBS,"""러시아, 여성 죄수들도 우크라 전장 내보내""",우크라이나 전쟁에 남성 죄수들을 용병으로 투입했던 러시아가 여성 죄수들까지 전장에 ...,https://n.news.naver.com/mnews/article/055/000...
1,세계,2023.03.15. 오후 4:00,조선비즈,[Why] “예금 전액 보호” 美 SVB 구제책에 비난 여론 불거진 이유,로이터=연합뉴스\n\n미국 실리콘밸리은행(SVB) 파산 사태에 조 바이든 미 행정부...,https://n.news.naver.com/mnews/article/366/000...
2,세계,2023.03.15. 오후 3:46,한국일보,냉전 후 첫 충돌한 미국·러시아 군용기...확전 피했지만 긴장 여전,"러 Su-27 전투기, 미 MQ-9 무인기 프로펠러 충돌\n미 무인기 흑해 해상 추...",https://n.news.naver.com/mnews/article/469/000...
3,세계,2023.03.15. 오후 3:45,조선일보,"“손 씻으면 싫어해”…日 유명 튀김집 직원, 1달만에 퇴사한 이유 폭로",일본의 한 '쿠시카츠 다나카' 매장. (기사 내용과는 관련 없는 매장 사진)/'쿠시...,https://n.news.naver.com/mnews/article/023/000...
4,세계,2023.03.15. 오후 4:33,데일리안,"美 법무·증권거래위원회, 'SVB 파산' 조사 착수","""예비조사 단계…기소·고발 안 이어질 듯""\n파산 전 경영진 지분 매각 조사 대상\...",https://n.news.naver.com/mnews/article/119/000...
5,세계,2023.03.15. 오후 4:22,MBC,"""러시아, 여성 죄수들도 우크라 전장 내보내""","포격당한 러시아 치하 우크라이나 볼노바카 마을, 군 조사관 모습 [자료사진]\n우크...",https://n.news.naver.com/mnews/article/214/000...
6,세계,2023.03.15. 오후 4:10,서울경제,"메타, 넉 달 만에 1만 명 또 자른다…""올해는 '효율의 해'""",감원 발표한 빅테크 중 첫 2차 구조조정\n우선순위 낮은 프로젝트 폐기 계획도\n2...,https://n.news.naver.com/mnews/article/011/000...
7,세계,2023.03.15. 오후 4:10,서울경제,"‘퇴출 위기’ 틱톡, 中 모기업과 분리 검토",“최후의 수단으로 ‘기업 분할’ 검토”\n글로벌 퇴출 우려에 자구책 고심\n틱톡. ...,https://n.news.naver.com/mnews/article/011/000...
8,세계,2023.03.15. 오후 3:57,SBS,[D리포트] 중국 공업용 세제 원료로 해삼 · 전복 가공,[중국 공업용 세제 원료로 해삼·전복 가공]\n\n중국 동북부 다롄의 한 수산물 업...,https://n.news.naver.com/mnews/article/055/000...
9,세계,2023.03.15. 오후 4:03,데일리안,"성병 갖고 태어났다...캐나다 '선천성 매독' 신생아 13배 급증, 이유는?",ⓒgettyimagesBank\n[데일리안 = 박상우 기자] 캐나다에서 매독에 걸린...,https://n.news.naver.com/mnews/article/119/000...


In [6]:
category = '정치'
csv_path = 'C:/myPyCode/news_csv/'
if f'{category}.csv' not in os.listdir(csv_path):
    old_link2 = 0
else:
    f = open(csv_path + f'{category}.csv', mode='r',  encoding = 'utf-8')
    file = csv.reader(f)
    old_link = list(file)
    old_link2 = [i[6] for i in old_link][1:]
    f.close()

for url in tqdm(politic_list):
    page = 1
    next_link_text = "다음"
    link_list = []
    while True:
        page_url = url + str(page)

        response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
        soup = bs4.BeautifulSoup(response.text, 'html.parser')
        link = soup.select('td.content li dt a')
        f_link_list = list(set([i.attrs['href'] for i in link]))
        link_list.append(f_link_list)
        addr = soup.find_all('a', {'class':'nclicks(fls.page)'})

        next_button = soup.find('a', {'class':'next'})
        if next_button:
            page += 1

        elif next_button is None:
            numlist = [i.get_text() for i in addr][1:]
            for page in numlist:
                page_url = url + str(page)
                response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
                soup = bs4.BeautifulSoup(response.text, 'html.parser')
                link = soup.select('td.content li dt a')
                f_link_list = list(set([i.attrs['href'] for i in link]))
                link_list.append(f_link_list)
            break

link_list = [j for i in link_list for j in i]
if old_link2 == 0:
    total_link = link_list
    additional_link = list(set(total_link))
else:
    total_link = old_link2 + link_list
    total_link = list(set(total_link))
    additional_link = [i for i in total_link if i not in old_link2]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:18<00:00,  3.07s/it]


In [8]:
len(additional_link)

212

In [9]:
len(old_link2)

456

In [7]:
len(total_link)

668